In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE129168'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High-throughput screening for modulators of CFTR activity applying an organotypic functional assay based on genetically engineered Cystic Fibrosis disease-specific iPSCs"
!Series_summary	"Organotypic culture systems from disease-specific induced pluripotent stem cells (iPSCs) exhibit obvious advantages compared to immortalized cell lines and primary cell cultures but implementation of iPSC-based high throughput (HT) assays is still technically challenging. Here we demonstrate the development and conduction of an organotypic HT Cl-/I- exchange assay using Cystic Fibrosis (CF) disease-specific iPSCs. The introduction of a halide sensitive YFP variant enabled automated quantitative measurement of Cystic Fibrosis Transmembrane Conductance Regulator (CFTR) function in iPSC-derived intestinal epithelia. CFTR function was partially rescued by treatment with VX-770 and VX-809, and seamless gene correction of the p.Phe508del mutation resulted in full resto

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True  # Based on the summary, gene expression is likely available

trait_row = 2  # CF status is inferred from 'genotype' in row 2
age_row = None
gender_row = None

def convert_trait(value):
    try:
        trait_value = value.split(':')[-1].strip()
        if "CFTR" in trait_value or "CFiPS" in trait_value:
            return 1 if "p.Phe508del" in trait_value else 0
        return None
    except Exception:
        return None

def convert_age(value):
    return None  # Age data is not available

def convert_gender(value):
    return None  # Gender data is not available

# Save cohort information
save_cohort_info('GSE129168', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cystic_Fibrosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cystic_Fibrosis/trait_data/GSE129168.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3701915': [1], 'GSM3701916': [1], 'GSM3701917': [1], 'GSM3701918': [1], 'GSM3701919': [1], 'GSM3701920': [1], 'GSM3701921': [1], 'GSM3701922': [1], 'GSM3701923': [1], 'GSM3701924': [1], 'GSM3701925': [1], 'GSM3701926': [1], 'GSM3701927': [0], 'GSM3701928': [0], 'GSM3701929': [0], 'GSM3701930': [0], 'GSM3701931': [0], 'GSM3701932': [0], 'GSM3701933': [0], 'GSM3701934': [0], 'GSM3701935': [0], 'GSM3701936': [0], 'GSM3701937': [0], 'GSM3701938': [0], 'GSM3701939': [0], 'GSM3701940': [0], 'GSM3701941': [0], 'GSM3701942': [0], 'GSM3701943': [0], 'GSM3701944': [0], 'GSM3701945': [0], 'GSM3701946': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'GB_ACC': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'LOCUSLINK_ID': [400451.0, 9899.0, 348093.0, 57099.0, 57082.0], 'GENE_SYMBOL': ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'KNL1'], 'GENE_NAME': ['family with sequence similarity 174 member B', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein, mRNA processing factor 2', 'apoptosis and caspase activation inhibitor', 'kinetochore scaffold 1'], 'UNIGENE_ID': ['Hs.27373', 'Hs.21754', 'Hs.436518', 'Hs.555966', 'Hs.181855'], 'ENSEMBL_ID': ['ENST00000553393', nan, nan, 'ENST00000306730', 'ENST00000527044'], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': ['ref|NM_207446|ens|ENST00000553393|ens|ENST00000327355|ref|XR_93

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys store gene identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Cystic_Fibrosis/gene_data/GSE129168.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Cystic_Fibrosis')

# 4. Save the cohort information.
save_cohort_info('GSE129168', './preprocessed/Cystic_Fibrosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Cystic_Fibrosis/GSE129168.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Cystic_Fibrosis', the least common label is '1.0' with 12 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.

